# Boss Fight: Ops Center Long Running

    - Goal: Manage long-running queue with retries
- Goal: Summarise SLO metrics

    **Pass criteria:** SLO summary stored after processing queue of five tasks.

    _Mock mode uses fixtures and writes artifacts under `_runs/`. Flip `USE_REAL_CLIENT=True` when ready for live calls._

In [ ]:

USE_REAL_CLIENT = False  # flip when ready
import os, json, time, pathlib, math, random

RUN = pathlib.Path('_runs/boss_ops_center_long_running')
RUN.mkdir(parents=True, exist_ok=True)
FIXTURE_ROOT = pathlib.Path(os.environ.get('MOCK_FIXTURE_ROOT', '_fixtures'))

def write_meta(**kwargs):
    payload = {"timestamp": time.time(), **kwargs}
    (RUN / 'meta.json').write_text(json.dumps(payload, indent=2))
    return payload

USE_REAL_CLIENT = bool(str(os.environ.get('USE_REAL_CLIENT', USE_REAL_CLIENT)).lower() in {'1', 'true', 'yes'})


In [ ]:
from tools.backoff import retry
tasks = [{"id": f"task-{i}", "should_fail": i % 2 == 0} for i in range(1, 6)]
results = []
for task in tasks:
    attempts = {"count": 0}
    def process():
        attempts["count"] += 1
        if task["should_fail"] and attempts["count"] < 2:
            raise RuntimeError("transient")
        return "done"
    result, history = retry(process, attempts=3, sleeper=lambda _: None)
    results.append({"task": task["id"], "result": result, "attempts": len(history)})
slo = {
    "processed": len(tasks),
    "failures": sum(1 for item in results if item["result"] is None),
    "max_attempts": max(item["attempts"] for item in results)
}
(RUN / "slo_summary.json").write_text(json.dumps({"results": results, "slo": slo}, indent=2))
slo

In [ ]:
meta = write_meta(ok=True, lab="boss_ops_center_long_running", max_attempts=slo['max_attempts'])
meta